In [53]:
import math
import pandas as pd
from scipy.optimize import minimize_scalar
#data
L= 0.3 # module length in meters
n_f = 6 #number of fibers
i_d = 1.27e-3#internal diameter m
p_s = 0.1e-6 #pore size in meters
T = 298 #K
gamma_W = 32400 #shear rate s-1
gamma_w_list = range(20000,90000,500)
phi_m = 0.68
eta_0 = 0.000905#Pa.second viscosity of hte TGM
rho_p = 1300 #kg/m3 density of the particles
rho_f = 1000 #kg/m3 density of the fluid
k_B = 1.38064852e-23 #J/mol/K Boltzmann constant
phi_w_guess = [0.64,0.64,0.64] #guess for the volume fraction of solute at the membrane wall
# Particle sizes (in meters)
particles = [
    {'name': '10nm', 'radius': 10e-9, 'phi_b': 0.01},
    {'name': '180nm', 'radius': 180e-9, 'phi_b': 0.05},
    {'name': '300nm', 'radius': 300e-9, 'phi_b': 0.06}
]
phi_w_dict={key:values for key, values in zip([particle['radius'] for particle in particles], phi_w_guess)}
target_species = 10e-9 
#equations
#Step 2 - evaluate the viscosity
    # no particle-particle interactions - dillute suspensions
def viscosity_no_PP(eta_0,phi_b,k1):
    """eta_0 - viscosity of the dispersion medium
    phi - volume in parts occupied by the dispersed solid
    k1 = shape factor"""
    eta_phi = eta_0*(1+5/2*phi_b+k1*phi_b**2)
    return eta_phi
eta_f = viscosity_no_PP(eta_0,sum(particles['phi_b'] for particles in particles),10)

# eta_f = 0.001307
diffusion_coeff = (k_B * T) / (6 * math.pi * eta_f * target_species)


#k1 around 10 for spheres
#not sure about the shape factor

def max_agg_packing (phi_m):
    phi_Max=phi_m+0.74*(1-phi_m)
    return phi_Max
phi_M = max_agg_packing(phi_m)
# phi_M = 0.92

#step 3 - maximum back-transport velocity, u (m/s)- Brownian, shear-infuced and intertial lift - Full retention of all solutes
 #particle volume franction at the membrane wall for each particle size for the first iteration
#J_flux = 30 #LMH


def J_brownian(a, phi_w, phi_b,gamma_w=gamma_W):
    numerator = gamma_w * (k_B ** 2) * (T ** 2)
    denominator = (eta_f ** 2) * (a ** 2) * L
    term = (numerator / denominator) ** (1/3)
    J = 0.114 * term * math.log(phi_w / phi_b)
    return J

# Function to calculate J for shear-induced diffusion
def J_shear(a, phi_w, phi_b,gamma_w=gamma_W):
    term = (a ** 4 / L) ** (1/3)
    log_term = math.log(phi_w / phi_b)
    J = 0.078 * term * gamma_w * abs(log_term)
    return J

# Function to calculate J for inertial lift
def J_inertial(a,gamma_w=gamma_W):
    J = (0.036 * rho_p * (a ** 3) * (gamma_w ** 2)) / eta_f
    return J
#J solvent permeation flux (m/s)


set_data = pd.DataFrame(columns=['name', 'radius', 'max_velocity', 'source'])

def min_flux(Particles, phi_w_list,gamma_w=gamma_W):
    """ have to put particles list with radius and phi_b,
    phi_w_list with phi_w
    results ai_target, J_flux and dataframe"""
    
    min_velocity = float('inf')
    selected_particle_radius = None
    velocities_list=[]

    for particle, phi_w in zip(Particles, phi_w_list):
        a = particle['radius']
        phi_b = particle['phi_b']
        
        # Calculate velocities
        brownian_velocity = J_brownian(a, phi_w, phi_b,gamma_w=gamma_W)
        shear_velocity = J_shear(a, phi_w, phi_b,gamma_w=gamma_W)
        inertial_velocity = J_inertial(a,gamma_w=gamma_W)
        
        # Find the maximum velocity
        max_velocity = max(brownian_velocity, inertial_velocity, shear_velocity)
        
        # Determine the source of the maximum velocity
        source = 'brownian' if max_velocity == brownian_velocity else 'inertial' if max_velocity == inertial_velocity else 'shear'
        
        # Update set_data by replacing previous row or adding a new one
        if particle['name'] in set_data['name'].values:
            set_data.loc[set_data['name'] == particle['name'], ['radius', 'max_velocity', 'source']] = [a, max_velocity, source]
        else:
            set_data.loc[len(set_data)] = [particle['name'], a, max_velocity, source]
        # Store all velocities in the DataFrame
        velocities_list.append({
            'name': particle['name'],
            'radius': a,
            'brownian_velocity': brownian_velocity,
            'shear_velocity': shear_velocity,
            'inertial_velocity': inertial_velocity
        })
        velocities_df=pd.DataFrame(velocities_list)
        
        # Check for the minimum velocity
        if max_velocity < min_velocity:
            min_velocity = max_velocity
            selected_particle_radius = a

    return selected_particle_radius, min_velocity, set_data,velocities_df

# Initialize variables for particles and inertial-lift properties
# ai_target, min_vel, dataf= min_flux(particles, phi_w_guess)

# Function to calculate φ_w for each particle size
def calculate_phi_w(Particles, a_target, min_velocity, data):
    count = 0
    # Dictionaries to store results
    phi_w_J_dict = {}
    phi_wjI_dict = {}
    inertial_particles = []
    
    for particle in Particles:
        a_particle = particle['radius']
        phi_b_particle = particle['phi_b']
        
        if a_particle != a_target:
            # Retrieve the velocity source from the data DataFrame
            source = data.loc[data['radius'] == a_particle, 'source'].values[0]
            
            # Use the source to choose the max_velocity
            if source == 'inertial':
                max_velocity = J_inertial(a_particle)
            elif source == 'shear':
                max_velocity = J_shear(a_particle, phi_w_dict.get(a_particle, 0.64), phi_b_particle)
            elif source == 'brownian':
                max_velocity = J_brownian(a_particle, phi_w_dict.get(a_particle, 0.64), phi_b_particle)
            else:
                raise ValueError(f"Unknown source {source} for particle {a_particle}")

            if max_velocity >= 10 * min_velocity and source == 'inertial':
                # Handle inertial particles
                phi_wjI = 0
                phi_wjI_dict[a_particle] = phi_wjI
                phi_w_dict[a_particle] = phi_wjI
                phi_w_J_dict[a_particle] = phi_wjI
                inertial_particles.append((a_particle, phi_b_particle, max_velocity))
                count += 1
            else:
                # Define the objective function based on the source
                def objective_function(phi_w_particle):
                    # Recalculate velocities based on the current wall concentration phi_w_particle
                    if source == 'brownian':
                        max_velocity = J_brownian(a_particle, phi_w_particle, phi_b_particle)
                    elif source == 'shear':
                        max_velocity = J_shear(a_particle, phi_w_particle, phi_b_particle)
                    elif source == 'inertial':
                        max_velocity = J_inertial(a_particle)
                    return abs(max_velocity - min_velocity)

                # Minimize the objective function to find the optimal phi_w for the particle
                result = minimize_scalar(objective_function, bounds=(0, 0.64), method='bounded')

                if result.success:
                    phi_w_optimal = result.x
                    phi_w_dict[a_particle] = phi_w_optimal
                    phi_w_J_dict[a_particle] = phi_w_optimal
                else:
                    pass
                    # print(f"Optimization failed for particle radius {a_particle:.1e} m")
    
    # Handle inertial particles after processing all particles
    if count == 1:
        for a, phi_b, u_j in inertial_particles:
            phi_wjI = phi_M - sum(phi_w_J_dict.values())
            phi_wjI_dict[a] = phi_wjI
            phi_w_dict[a] = phi_wjI
    else:
        total_u_j_inv = sum((phi_b / u_j) for _, phi_b, u_j in inertial_particles)
        phi_w_remaining = phi_M - sum(phi_w_J_dict.values())
        phi_w_jI_sum = sum((phi_b / u_j) / total_u_j_inv * phi_w_remaining for _, phi_b, u_j in inertial_particles)
        if abs(phi_M - (sum(phi_w_J_dict.values()) + phi_w_jI_sum)) > 1e-6:
            # print("Warning: φ_M and the sum of φ_w_j do not match closely enough.")
            pass
        for a, phi_b, u_j in inertial_particles:
            phi_wjI = (phi_b / u_j) / total_u_j_inv * phi_w_remaining
            phi_wjI_dict[a] = phi_wjI
            phi_w_dict[a] = phi_wjI

    return phi_w_dict


# Output the updated dictionary
#print(f"Updated phi_w_dict: {phi_w_dict}")
def packing_constraints(Particles, phi_w_list,gamma_w=gamma_W):
    ai_target, min_vel, dataf,vel= min_flux(Particles, phi_w_list,gamma_w=gamma_W)
    phi_w_updated=calculate_phi_w(Particles, ai_target, min_vel, dataf)
    
    tolerance = 1e-6  # Set a tolerance for convergence
    max_iterations = 1000
    smallest_key = min(phi_w_updated.keys())
    phi_smallest_int = 0
    r_minimum = 0

    for i in range(max_iterations):
        remaining_phiw_j = {a: phi_w for a, phi_w in phi_w_updated.items() if a != smallest_key}
        phi_w_i = phi_w_updated[ai_target] # Wall concentration of the target particle
        # Check if packing constraints are satisfied
        if (phi_M >= sum(phi_w_updated.values())) and (sum(remaining_phiw_j.values()) <= 0.68) and phi_w_updated[smallest_key] <= 0.74 * (1 - sum(remaining_phiw_j.values())) and phi_smallest_int<0.74:
            # print(f"Iteration {i}: Packing constraints are met.")
            return phi_w_updated,   min_vel, ai_target, dataf,r_minimum, phi_smallest_int
        
        # Store the previous values of phi_w for comparison
        phi_w_previous = phi_w_updated.copy()
        # Update phi_w for the target particle
        phi_w_i_corrected = phi_M * (phi_w_i / sum(phi_w_updated.values()))
        phi_w_updated[ai_target] = phi_w_i_corrected
        phi_w_updated_list = list(phi_w_updated.values())
        #print(phi_w_updated, J_flux, a_target)
        # Recalculate the flux and phi_w_j values after updating
        ai_target, min_vel, dataf,vel= min_flux(particles, phi_w_updated_list)
        phi_w_updated=calculate_phi_w(particles, ai_target, min_vel, dataf)
        phi_smallest_int= phi_w_dict[min(phi_w_dict.keys())] / (1 -sum(phi_w for phi_w in phi_w_dict.values() if phi_w != phi_w_dict[min(phi_w_dict.keys())]))
#        
#       # #step - 9 - calculate the minimum
        r_minimum = (min(phi_w_dict.keys()) * ( (math.sqrt(2) * ((( 4 * (4 / 3) * math.pi ) / phi_smallest_int )) ** (1/3)) - 2 ))/2
        
        # Check for the change in phi_w values between iterations
        phi_w_diff = sum(abs(phi_w_updated[a] - phi_w_previous[a]) for a in phi_w_updated)

        # If the change in phi_w values is smaller than the tolerance, stop the iteration
        if phi_w_diff < tolerance:
            # print(f"Iteration {i}: Will not converger further.")
            return phi_w_updated,   min_vel, ai_target, dataf,r_minimum, phi_smallest_int
    # If the loop completes without finding a solution
    print("Warning: No solution found within the iteration limit.")
  
    return phi_w_updated, min_vel, ai_target, dataf,r_minimum, phi_smallest_int

#mass-transfer coefficient calculated from equation  1#step 10


def sieving_parameters(Particles, phi_w_list,gamma_w=gamma_W):
    phi_w_updated, J_flux, ai_target, dataf, r_min, phi_smal_int = packing_constraints(Particles, phi_w_list,gamma_w=gamma_W)
    print(f"Updated phi_w_dict: {phi_w_updated}. J_flux: {J_flux}")
    S_0=0

    for i in range(50):
        Q = J_flux * n_f * math.pi * i_d * (L + L / 20)
        mass_transfer_k = 1.62 * (diffusion_coeff / i_d) * (((4 * rho_f * Q) / (math.pi * n_f * i_d * eta_f)) ** (1 / 3)) * ((eta_f / (rho_f * diffusion_coeff)) ** (1 / 3))
        print(f"mass transfer coefficient: {mass_transfer_k}")
        lambda_s_min = target_species / r_min
        print(f"lambda_s_min: {lambda_s_min}")
        Sieving_coeff_int = ((1 - lambda_s_min) ** 2) * (2 - ((1 - lambda_s_min) ** 2)) * math.exp(-0.7146 * (lambda_s_min ** 2))
        print(f"Initial Sieving coefficient: {Sieving_coeff_int}")
        delta_m = target_species * ((4 * (4 / 3) * math.pi) / phi_smal_int) ** (1 / 3)
        print(f"delta_m: {delta_m}")
        phi_e_K_d = (1 - lambda_s_min) ** (9 / 2)
        print(f"phi_e_K_d: {phi_e_K_d}")
        cake_membrane_porosity = 1 - phi_smal_int
        print(f"cake_membrane_porosity: {cake_membrane_porosity}")
        Peclet_m = ((J_flux * delta_m) / diffusion_coeff) * (Sieving_coeff_int / (cake_membrane_porosity * phi_e_K_d))
        print(f"Peclet_m: {Peclet_m}")
        Sieving_coeff_act = (Sieving_coeff_int * math.exp(Peclet_m)) / (Sieving_coeff_int + math.exp(Peclet_m) - 1)
        print(f"Actual Sieving coefficient: {Sieving_coeff_act}")
        Sieving_coeff_obs = Sieving_coeff_act / (((1 - Sieving_coeff_act) * math.exp(-J_flux / mass_transfer_k)) + Sieving_coeff_act)
        print(f"Observed Sieving coefficient: {Sieving_coeff_obs}")
        yields = 1 - math.exp(-4 * (Sieving_coeff_obs))
        print(f"Yields: {yields}")
        # Convergence check for the main loop
        if i==0 and yields>0.95:
            # print(f"Final iteration {i}")
            # print(f"Final Sieving coefficient (obs): {Sieving_coeff_obs}")
            # print(f"Final Yield: {yields}")
            # print(f"Final flux: {J_flux*3600*1000}")
            # for particle in Particles:
            #     name = particle['name']
            #     phi_b = particle['phi_b']
            #     phi_w = phi_w_updated[particle['radius']]
            #     print(f"Particle {name}:")
            #     print(f"  Bulk concentration (phi_b): {phi_b}")
            #     print(f"  Wall concentration (phi_w): {phi_w}")
            return dataf, phi_w_updated,yields
        if (abs(S_0-Sieving_coeff_obs)/((S_0+Sieving_coeff_obs)/2)) < 0.1:
            # print(f"Convergence reached after {i} iterations.")
            # print(f"Final Sieving coefficient (obs): {Sieving_coeff_obs}")
            # print(f"Final Yield: {yields}")
            # print(f"Final flux: {J_flux*3600*1000}")
            # for particle in Particles:
            #     name = particle['name']
            #     phi_b = particle['phi_b']
            #     phi_w = phi_w_updated[particle['radius']]
            #     print(f"Particle {name}:")
            #     print(f"  Bulk concentration (phi_b): {phi_b}")
            #     print(f"  Wall concentration (phi_w): {phi_w}")
            return dataf, phi_w_updated,yields
        
        if Sieving_coeff_obs<0.75: # and i==0:
        # # # Update flux if sieving coefficient is within a valid range
            # mass_transfer_k = J_flux/math.log((phi_w_updated[target_species] / (Particles[0]['phi_b'])))
            J_flux = mass_transfer_k * math.log(( 0.64/ (Particles[0]['phi_b'] * (1 - Sieving_coeff_obs))))
            print(f"Updated flux: {J_flux}, iteration {i}")

        S_0 = Sieving_coeff_obs
        # Sub-loop for calculating phi_w and r_min
        for j in range(50):
            phi_w_updated=calculate_phi_w(Particles, ai_target, J_flux, dataf)
            
            phi_w_i = phi_w_updated[ai_target] # Wall concentration of the target particle
            # Store the previous values of phi_w for comparison
            phi_w_previous = phi_w_updated.copy()
            # Update phi_w for the target particle
            phi_w_i_corrected = phi_M * (phi_w_i / sum(phi_w_updated.values()))
            phi_w_updated[ai_target] = phi_w_i_corrected
           
            # Recalculate the flux and
            ai_target, J_flux, dataf,vel= min_flux(Particles, phi_w_updated.values())
            phi_smal_int= phi_w_dict[min(phi_w_dict.keys())] / (1 -sum(phi_w for phi_w in phi_w_dict.values() if phi_w != phi_w_dict[min(phi_w_dict.keys())]))
        #       # #step - 9 - calculate the minimum
            r_min = (min(phi_w_dict.keys()) * ( (math.sqrt(2) * ((( 4 * (4 / 3) * math.pi ) / phi_smal_int )) ** (1/3)) - 2 ))/2
            remaining_phiw_j = {a: phi_w for a, phi_w in phi_w_updated.items() if a != min(phi_w_updated.keys())}
           
            if (phi_M >= sum(phi_w_updated.values())) and (sum(remaining_phiw_j.values()) <= 0.68) and phi_w_updated[min(phi_w_updated.keys())] <= 0.74 * (1 - sum(remaining_phiw_j.values())) and phi_smal_int<0.74:
                # print(f"Iteration {i}: Packing constraints are met.")
                break
            # Check for the change in phi_w values between iterations
            phi_w_diff = sum(abs(phi_w_updated[a] - phi_w_previous[a]) for a in phi_w_updated)
            
            # If the change in phi_w values is smaller than the tolerance, stop the iteration
            if phi_w_diff < 10e-6:
                # print(f"Iteration {i}: Will not converger further.")
                break

    print("Warning: No solution found within the iteration limit.")
    return dataf, phi_w_updated,yields

# print(sieving_parameters(particles, phi_w_guess))


In [55]:
data_figs=pd.DataFrame(columns=['shear_rate', 'J_pred_10',"J_pred_180","J_pred_300",'yield', 'phi_w_10', 'phi_w_180', 'phi_w_300'])


                  
for i in range(len(gamma_w_list)):
    phi_w_dict={key:values for key, values in zip([particle['radius'] for particle in particles], phi_w_guess)}
    gamma_W = gamma_w_list[i] #shear rate s-1
    print(gamma_W)
    data_F,phi_w_dict,Yield=sieving_parameters(particles, phi_w_guess,gamma_w=gamma_W)
    data_figs.loc[i] = [gamma_W, data_F.loc[0, 'max_velocity'], data_F.loc[1, 'max_velocity'], data_F.loc[2, 'max_velocity'], Yield, phi_w_dict[10e-9], phi_w_dict[180e-9], phi_w_dict[300e-9]]

   

20000
Updated phi_w_dict: {1e-08: 0.04408308960320423, 1.8e-07: 0.1320874656985093, 3e-07: 0.1330530393265679}. J_flux: 3.7271673430634863e-06
mass transfer coefficient: 1.3957772204008382e-06
lambda_s_min: 0.2760781004298786
Initial Sieving coefficient: 0.7324819247742471
delta_m: 6.536726214303966e-08
phi_e_K_d: 0.23367525031196515
cake_membrane_porosity: 0.9400115397505906
Peclet_m: 0.04864158272146131
Actual Sieving coefficient: 0.9829557339376109
Observed Sieving coefficient: 0.9988009992161951
Yields: 0.9815963082676549
20500
Updated phi_w_dict: {1e-08: 0.04752135209616655, 1.8e-07: 0.13874659623597624, 3e-07: 0.13572969578150437}. J_flux: 3.9158909482829275e-06
mass transfer coefficient: 1.4189486257591077e-06
lambda_s_min: 0.28663839631991755
Initial Sieving coefficient: 0.7155369744909151
delta_m: 6.348003244878468e-08
phi_e_K_d: 0.21872266295201728
cake_membrane_porosity: 0.9345006213121412
Peclet_m: 0.052100744557165304
Actual Sieving coefficient: 0.9802168508768058
Observed

OverflowError: math range error

In [ ]:
 0.000004 *1000*3600

In [56]:
data_figs

,shear_rate,J_pred_10,J_pred_180,J_pred_300,yield,phi_w_10,phi_w_180,phi_w_300
0,20000.0,0.000005,0.000006,0.000004,0.981596,0.044083,0.132087,0.133053
1,20500.0,0.000005,0.000006,0.000004,0.981591,0.047521,0.138747,0.135730
2,21000.0,0.000005,0.000006,0.000004,0.981584,0.051321,0.145918,0.138458
3,21500.0,0.000006,0.000006,0.000004,0.981576,0.055528,0.153644,0.141242
4,22000.0,0.000006,0.000007,0.000005,0.981567,0.060194,0.161978,0.144081
5,22500.0,0.000006,0.000007,0.000005,0.981555,0.065374,0.170972,0.146981
6,23000.0,0.000006,0.000007,0.000005,0.981541,0.071124,0.180683,0.149934
7,23500.0,0.000006,0.000007,0.000005,0.981522,0.077532,0.191180,0.152951
8,24000.0,0.000007,0.000007,0.000005,0.981498,0.084669,0.202530,0.156026
9,24500.0,0.000007,0.000007,0.000006,0.981466,0.092636,0.214813,0.159163


In [57]:
phi_w_dict={key:values for key, values in zip([particle['radius'] for particle in particles], phi_w_guess)}
sieving_parameters(particles, phi_w_guess,32400)

Updated phi_w_dict: {1e-08: 0.24366189597410762, 1.8e-07: 0.4970231686190307, 3e-07: 0.17286938754658857}. J_flux: 8.022827967095074e-06
mass transfer coefficient: 1.8021833321424652e-06
lambda_s_min: 0.9978829645087982
Initial Sieving coefficient: 4.399984058183018e-06
delta_m: 2.831427416774581e-08
phi_e_K_d: 9.242226865837573e-13
cake_membrane_porosity: 0.26187094376352205
Peclet_m: 247250.72705379885


OverflowError: math range error